# Classificazione

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import types
import matplotlib.pyplot as plt

%matplotlib inline

# Classificazione del dataset MNIST

Il dataset MINIST è costituito da immagini 28x28 pixel contenenti numeri scritti a mano. L'obiettivo è classificare ogni immagine con il numero corrispondente. Quindi serve addestrare un classificatore in grado di riconoscere 10 classi, i numeri da 1 a 10.

Le prossime celle del notebook eseguiranno le seguenti operazioni:

1. Caricamento del dataset
2. Split in train/test
3. Addestramento del modello
4. Valutazione dei risultati

In [26]:
args = types.SimpleNamespace()
args.log_interval = 10000
args.lr = 1e-3
args.epochs = 2

use_cuda = torch.cuda.is_available()

use_cuda

False

# Dataset

In [32]:
# download fix
new_mirror = 'https://ossci-datasets.s3.amazonaws.com/mnist'
datasets.MNIST.resources = [
   ('/'.join([new_mirror, url.split('/')[-1]]), md5)
   for url, md5 in datasets.MNIST.resources
]

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('./data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                   transform=transform)

batch_size = 1000
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

## Modello

In [33]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Train

In [34]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [35]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    y_pred = []
    y_true = []
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            y_true.extend(target.cpu().view(-1).tolist())
            y_pred.extend(pred.cpu().view(-1).tolist()) 

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return y_true, y_pred

In [36]:
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

try:
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        y_true, y_pred = test(model, device, test_loader)
except KeyboardInterrupt:
    pass

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301573

Test set: Average loss: 0.1143, Accuracy: 9650/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.192400

Test set: Average loss: 0.0676, Accuracy: 9788/10000 (98%)



# Stats

In [37]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       980
           1       0.97      1.00      0.99      1135
           2       0.98      0.97      0.98      1032
           3       0.99      0.98      0.98      1010
           4       0.97      0.99      0.98       982
           5       0.98      0.99      0.98       892
           6       0.99      0.98      0.99       958
           7       0.96      0.98      0.97      1028
           8       0.99      0.96      0.98       974
           9       0.99      0.95      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

